# TorchRec 学习笔记

> 记录 TorchRec 的核心概念、安装方式、关键组件与 API 索引，帮助快速构建与部署大规模推荐系统。TorchRec 深度集成 PyTorch 与 FBGEMM，面向稀疏特征、巨大嵌入表与分布式训练场景。

## 入门指南
- **TorchRec 概述**：PyTorch 官方推荐系统库，提供嵌入表组件、分片与分布式训练工具，是 Meta 内部推荐模型的核心基础设施。
- **为什么需要 TorchRec**：
  - 专用组件（EmbeddingBagCollection 等）让推荐模型开发更高效。
  - 复杂分片策略（表、列、行、网格等）应对海量嵌入表。
  - 结合 FBGEMM 的高性能实现，兼顾训练与推理。
  - 无缝嵌入 PyTorch 生态，沿用已有代码与工具链。
- **官方入门资源**：
  - TorchRec 概览：`overview.html`
  - 安装指南：`setup-torchrec.html`
  - 交互式教程（Colab）：<https://colab.research.google.com/github/pytorch/torchrec/blob/main/Torchrec_Introduction.ipynb>

## 安装与环境
- **支持环境**：Python 3.9–3.12；CPU、CUDA 11.8/12.1/12.4。
- **核心依赖**：PyTorch、FBGEMM；版本需匹配（如 TorchRec 1.0 ↔ FBGEMM 1.0 ↔ PyTorch 2.5）。
- **安装示例（CUDA 12.1）**：
  ```bash
  pip install torch --index-url https://download.pytorch.org/whl/cu121
  pip install fbgemm-gpu --index-url https://download.pytorch.org/whl/cu121
  pip install torchmetrics==1.0.3
  pip install torchrec --index-url https://download.pytorch.org/whl/cu121
  ```
- 如需源码编译或其他 CUDA 版本，替换 `cu121` 为 `cpu / cu118 / cu124`。

In [ ]:
# 检查 TorchRec 与依赖安装状态
try:
    import torch, torchrec
    from torchrec.fx import tracer  # 触发依赖导入
    print('PyTorch 版本:', torch.__version__)
    print('TorchRec 版本:', torchrec.__version__)
except Exception as err:
    print('TorchRec 未安装或导入失败 ->', err)
    print('请按文档安装对应版本的 torch / fbgemm-gpu / torchrec')

## TorchRec 核心概念
### 稀疏特征数据结构
- **JaggedTensor**：通过 `lengths/offsets + values` 存放单一稀疏特征，避免 padding。
- **KeyedJaggedTensor (KJT)**：带键的 JaggedTensor，支持一次传入多组特征。
- **KeyedTensor**：将多路 dense 张量按键连接，常用于池化后的结果。

除了数据结构，TorchRec 训练流程包含以下组件：
1. **Planner**：根据嵌入表配置、硬件拓扑生成最优分片方案。
2. **Sharder**：按方案对模型进行表/行/列等多策略分片。
3. **DistributedModelParallel (DMP)**：整合分片模型、优化器与数据并行入口。
4. **Sparse Optimizer**：将梯度回传与优化一步融合，进一步提升效率。

### JaggedTensor 示例
```python
# 用户交互记录：三位用户分别交互 2/3/1 个物品
lengths = torch.tensor([2, 3, 1])
values = torch.tensor([101, 102, 201, 202, 203, 301])
jt = JaggedTensor(lengths=lengths, values=values)
```
- `lengths` 表示每个样本的元素个数。
- `offsets` 可替代 `lengths` 存储起始索引。
- `values` 为稀疏特征真实内容。

`KeyedJaggedTensor` 支持多个键：
```python
kjt = KeyedJaggedTensor.from_jt_dict({
    'user': JaggedTensor(values=torch.tensor([1,2,3]), lengths=torch.tensor([2,1])),
    'item': JaggedTensor(values=torch.tensor([10,11]), lengths=torch.tensor([1,1]))
})
```

## 训练流程与分片策略
1. **Planner 规划**：根据嵌入表元信息、硬件拓扑、约束，生成 `ShardingPlan`。
2. **Sharder 执行**：常见策略包括：
   - Table-Wise (TW)：整表放单卡。
   - Row-Wise (RW)：按行切分，适合大表。
   - Column-Wise (CW)：按 embedding 维度切分。
   - Table-Wise-Row-Wise / Grid-Shard：组合策略。
   - Data-Parallel (DP)：每卡保留一份副本。
3. **DistributedModelParallel**：封装分片模块，结合数据并行训练。
4. **通信流程**：`input_dist -> lookup -> output_dist`，对应特征分发、嵌入查找、结果回传；反向时反序执行。

### TorchRec Planner 组件
- **EmbeddingShardingPlanner**：主入口，组合 Enumerator / Proposer / Partitioner / Estimator。
- **Enumerator**：枚举可行分片选项。
- **Proposer**：贪心/启发式筛选最优组合。
- **Partitioner**：实际放置分片，考虑内存与性能。
- **StorageReservation / PerfEstimator**：估计存储与时间开销。

## 示例：EmbeddingBagCollection 前向

In [ ]:
import torch
from torchrec.sparse.jagged_tensor import JaggedTensor, KeyedJaggedTensor
import torchrec
from torchrec.modules.embedding_configs import EmbeddingBagConfig
from torchrec.modules.embedding_modules import EmbeddingBagCollection

# 定义两个嵌入表
product_cfg = EmbeddingBagConfig(
    name='product_table',
    embedding_dim=16,
    num_embeddings=4096,
    feature_names=['product'],
)
user_cfg = EmbeddingBagConfig(
    name='user_table',
    embedding_dim=16,
    num_embeddings=4096,
    feature_names=['user'],
)

# 创建 EmbeddingBagCollection
ebc = EmbeddingBagCollection(device='cpu', tables=[product_cfg, user_cfg])

# 构建样例 JaggedTensor
device = torch.device('cpu')
product_jt = JaggedTensor(values=torch.tensor([1, 2, 1, 5], device=device),
                          lengths=torch.tensor([3, 1], device=device))
user_jt = JaggedTensor(values=torch.tensor([2, 3, 4, 1], device=device),
                       lengths=torch.tensor([2, 2], device=device))

kjt = KeyedJaggedTensor.from_jt_dict({'product': product_jt, 'user': user_jt})
output = ebc(kjt)
print('输出 KeyedTensor keys:', output.keys())
print('输出形状:', output.values().shape)

## 分布式训练与模型并行
- **DistributedModelParallel**：主入口，接受原始 `nn.Module`、分片计划与 sharders，自动包装分布式训练环境。
- 支持延迟初始化 (`init_data_parallel=False`)、自定义数据并行封装、监控追踪等。
- 训练过程可通过 `reshard` 动态调整分片策略。

## 推理部署
- **量化**：`quantize_inference_model` 将训练模块替换为量化版本（如 `QuantEmbeddingBagCollection`）。
- **分片推理**：`shard_quant_model` 按需在多设备上分布推理模型，生成新的 `ShardingPlan`。
- **C++ 部署**：模型可通过 TorchScript/Fx 导出，结合 FBGEMM 在 C++ 环境执行，进一步降低延迟。

## 关键模块速览
- **数据类型**：`JaggedTensor`、`KeyedJaggedTensor`、`KeyedTensor`。
- **嵌入模块**：`EmbeddingBagCollection`、`EmbeddingCollection` 及对应 Sharded/Quant 版本。
- **Planner 相关**：`EmbeddingShardingPlanner`、`EmbeddingEnumerator`、`GreedyPerfPartitioner`、`HeuristicalStorageReservation`、`GreedyProposer`、`EmbeddingPerfEstimator` 等。
- **分布式**：`DistributedModelParallel`、`ShardingPlan`、`EmbeddingModuleShardingPlan`。
- **推理**：`torchrec.inference.modules.quantize_inference_model`、`shard_quant_model`。

## API 参考索引
- `torchrec.sparse.jagged_tensor`
- `torchrec.modules.embedding_configs`
- `torchrec.modules.embedding_modules`
- `torchrec.distributed.types / planner / model_parallel`
- `torchrec.inference.modules`
- 更多内容参阅官方文档与 Sphinx 页面。

## 进一步阅读
- TorchRec 官方文档：<https://pytorch.org/torchrec/latest/>
- Interactive Notebook：TorchRec Introduction（Colab）
- FBGEMM GPU：<https://github.com/pytorch/FBGEMM>
- TorchRec GitHub：<https://github.com/pytorch/torchrec>